## Imports and settings

In [30]:
import ast
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('wordnet')

import spacy
nlp = spacy.load('en_core_web_md')

pd.options.display.max_colwidth = None

print('DONE!')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anastasiiatodoshchuk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


DONE!


## Loading data (with annotation)

In [31]:
annotate_1 = pd.read_csv("fairy_tales/Cinderella_annotate.csv", sep=';', encoding='utf-8')
annotate_1 = annotate_1.drop(['Unnamed: 0'], axis=1)

annotate_2 = pd.read_csv("fairy_tales/Pied_Piper_annotate.csv", sep=';', encoding='utf-8')
annotate_2 = annotate_2.drop(['Unnamed: 0', ], axis=1)

annotate_3 = pd.read_csv("fairy_tales/Snow_White_annotate.csv", sep=';', encoding='utf-8')
annotate_3 = annotate_3.drop(['Unnamed: 0'], axis=1)

annotate_4 = pd.read_csv("fairy_tales/The_Little_Mermaid_annotate.csv", sep=';', encoding='utf-8')
annotate_4 = annotate_4.drop(['Unnamed: 0'], axis=1)

annotate_5 = pd.read_csv("fairy_tales/The_Ugly_Duckling_annotate.csv", sep=';', encoding='utf-8')
annotate_5 = annotate_5.drop(['Unnamed: 0'], axis=1)

frames = [annotate_2, annotate_3, annotate_4, annotate_5]

annotate = annotate_1.append(frames).reset_index(drop=True)
annotate = annotate.drop(["Sentence_ID", "Annotate_Purpose", "Annotate_Cause", "Annotate_Complement"], axis=1)
annotate.head()

,Sentence_content,Annotate_Action,Annotate_Agent,Annotate_Direct_Obj,Annotate_Indirect_Obj,Annotate_Location,Annotate_Time
0,"Once upon a time, there was a beautiful girl named Cinderella.","was, named",NaN,Cinderella,NaN,NaN,Once upon a time
1,She lived with her wicked stepmother and two stepsisters.,lived,She,NaN,NaN,with her wicked stepmother and two stepsisters,NaN
2,They treated Cinderella very badly.,treated,They,Cinderella,NaN,NaN,NaN
3,"One day, they were invited for a grand ball in the king’s palace.",were invited,they,NaN,NaN,in the king’s palace,One day
4,But Cinderella’s stepmother would not let her go.,would not let go,Cinderella’s stepmother,her,NaN,NaN,NaN


## Splitting complex sentences into simple sub-sentences (Spacy)

In [32]:
def get_sub_sentences(data):

    data['sub_sentences'] = None
    previous_token = ''
    sub_sentences = []
    sub_sentence = ''

    for row in range(data.shape[0]):

        doc_sent = nlp(data.Sentence_content[row])
        #doc_sent = nlp(data.sent_token[row])
        for token in doc_sent:

            # token is a part of the next sentence
            # CCONJ is 'and', 'but'
            if token.pos_ == 'CCONJ' and previous_token:

                # exammple: ..., and ..."
                if previous_token.pos_ == 'PUNCT':
                    sub_sentences.append(sub_sentence)
                    sub_sentence = ''
                    sub_sentence = sub_sentence + token.text + ' '

            # SCONJ is 'because', 'that', 'so'
            elif token.pos_ == 'SCONJ' and previous_token:

                # token is a part of the current sentence
                if previous_token.pos_ == 'SCONJ':
                    sub_sentence = sub_sentence + token.text + ' '

                # token is a part of the current sentence
                if previous_token.pos_ == 'CCONJ':
                    sub_sentence = sub_sentence + token.text + ' '

                # token is a part of the current sentence (Row 3: "ones upon a time..." case)
                if len(sub_sentence.split())==1:
                    sub_sentence = sub_sentence + token.text + ' '

                # token is a part of the new sentence
                else:
                    sub_sentences.append(sub_sentence)
                    sub_sentence = ''
                    sub_sentence = sub_sentence + token.text + ' '

            # token is a part of the current sentence
            elif token.text == ';':
                sub_sentence = sub_sentence + '.'
                sub_sentences.append(sub_sentence)
                sub_sentence = ''

            else:
                sub_sentence = sub_sentence + token.text + ' '

            previous_token = token

        sub_sentences.append(sub_sentence)
        data.loc[row, 'sub_sentences'] = str(sub_sentences)
        previous_token = ''
        sub_sentences = []
        sub_sentence = ''


    data['sub_sentences'] = data['sub_sentences'].apply(ast.literal_eval)

    data['count_sub'] = None
    for row in range(data.shape[0]):
        data.loc[row, 'count_sub'] = len(data.sub_sentences[row])

    print('DONE!')

    return data

#### Run sub-sentences algorithm on annotate dataframe

In [33]:
data_sub = annotate.copy()
data_sub = get_sub_sentences(data_sub)
print("Example:")
data_sub.iloc[185:188, [0,7,8]]

DONE!
Example:


,Sentence_content,sub_sentences,count_sub
185,A little duckling was very sad because he thought he was the ugliest amongst all his brothers and sisters.,"[A little duckling was very sad , because he thought he was the ugliest amongst all his brothers sisters . ]",2
186,They would not play with him.,[They would not play with him . ],1
187,They teased the poor ugly duckling.,[They teased the poor ugly duckling . ],1


# 5 WH questions

## Q1, Q2, and Q3: agent (who?), event (what did agent do?), and patient (to whom?).

In [12]:
# This function is looking for agents, actions, and patients on the level of sub-sentences.
def get_agents_actions_patients(data):
    agent_roles = ['nsubj', 'nsubjpass']
    verb_roles = ['ROOT', 'conj', 'ccomp', 'advcl', 'relcl']
    patient_roles = ['dobj', 'attr']

    annex_agent_roles = ['conj', 'compound', 'advcl']

    data.insert(loc=2, column='actions_in_sent', value=None)
    data.insert(loc=4, column='agents_in_sent', value=None)
    data.insert(loc=7, column='patients_in_sent', value=None)

    agents_in_sent = []
    actions_in_sent = []
    patients_in_sent = []
    event = ''
    agent = ''
    patient = None

    final_agents = []
    final_actions = []
    final_patients = []

    for row in range(data.shape[0]):
        for subsent in data['sub_sentences'][row]:
            doc = nlp(subsent)
            for word in doc:

                # -- this is our main AGENT (word) and main EVENT (word.head) --
                if (word.dep_ in agent_roles) and (word.head.dep_ in verb_roles):

                    agent = word.text
                    event = word.head.text

                    agents_in_sent.append(agent)
                    actions_in_sent.append(event)

                    # -- are there a PATIENT for the found AGENT and EVENT pair? --
                    for echild in word.head.children:
                        # -- if there's a patient and if it's the 1st one - add it
                        # -- (it goes in set with main agent and event).
                        if echild.dep_ in patient_roles:
                            patient = echild.text
                            patients_in_sent.append(patient)

                    # -- if no patient was found, than add the None. If it was found, make it None
                    if patient:
                        patient = None
                    else:
                        patients_in_sent.append(patient)

                    # -- different agent-children with the same parent event --
                    for achild in word.children:
                        if achild.dep_ in annex_agent_roles:
                            agent = achild.text
                            agents_in_sent.append(agent)
                            actions_in_sent.append(event)

                    # -- different event-children with the same parent agent --
                    agent = word.text
                    for echild in word.head.children:
                        if echild.dep_ in annex_agent_roles:
                            event = echild.text
                            agents_in_sent.append(agent)
                            actions_in_sent.append(event)

                    # -- combinations agent-children with event-children --
                    for achild in word.children:
                        if achild.dep_ in annex_agent_roles:
                            agent = achild.text
                            for echild in word.head.children:
                                if echild.dep_ in annex_agent_roles:
                                    event = echild.text
                                    agents_in_sent.append(agent)
                                    actions_in_sent.append(event)


            final_agents.append(agents_in_sent)
            final_actions.append(actions_in_sent)
            final_patients.append(patients_in_sent)

            agents_in_sent = []
            actions_in_sent = []
            patients_in_sent = []

        data['agents_in_sent'][row] = final_agents
        data['actions_in_sent'][row] = final_actions
        data['patients_in_sent'][row] = final_patients

        final_agents = []
        final_actions = []
        final_patients = []

    print('DONE!')

    return data

In [34]:
data_1 = data_sub.copy()
data_1 = get_agents_actions_patients(data_1)
print("Example:")
data_1.iloc[185:188, [0,2,4,7,10,11]]

DONE!
Example:


,Sentence_content,actions_in_sent,agents_in_sent,patients_in_sent,sub_sentences,count_sub
185,A little duckling was very sad because he thought he was the ugliest amongst all his brothers and sisters.,"[[was], [thought, was]]","[[duckling], [he, he]]","[[None], [None, ugliest]]","[A little duckling was very sad , because he thought he was the ugliest amongst all his brothers sisters . ]",2
186,They would not play with him.,[[play]],[[They]],[[None]],[They would not play with him . ],1
187,They teased the poor ugly duckling.,[[teased]],[[They]],[[duckling]],[They teased the poor ugly duckling . ],1


## Q4: time (When?) (Special Version for this Fairy tale!)
### 1. Extracting clauses with "when" conjunction

In [35]:
# collects words from whole WHEN clauses to when_list

def extract_all_children_when(doc, head):
    if head.children:
        for child in head.children:
            when_list.append(child.text)
            extract_all_children_when(doc, child)

In [36]:
def get_when_clauses(data):

    global when_list
    when_list = []
    when_str = ''

    data['time_when'] = None

    # looking for clauses which start with when in each row
    for row in range(data.shape[0]):
        doc = nlp(data.iloc[row,0])

        for word in doc:

            # check if word belongs to WHEN collocation (works if when_list is not empty)
            if word.text in when_list:
                when_str = when_str + ' ' + str(word.text)
            else:
                # if doesn't belong, then collocation is over, clean it
                when_list = []

            # if word = 'when' then run the function that collects words from whole WHEN collocation
            if word.text == 'when':
                when_list.append(word.text)
                head = word.head
                when_list.append(head.text)
                extract_all_children_when(doc, head)
                when_str = 'when'

        data['time_when'][row] = when_str
        when_str = ''
    print('DONE!')

    return data

In [37]:
data_2 = annotate.copy()
data_2["Sentence_content"] = data_2["Sentence_content"].str.lower()
data_2["Annotate_Time"] = data_2["Annotate_Time"].str.lower()
data_2 = get_when_clauses(data_2)
print("Example:")
data_2.iloc[25:30, :]


DONE!
Example:


,Sentence_content,Annotate_Action,Annotate_Agent,Annotate_Direct_Obj,Annotate_Indirect_Obj,Annotate_Location,Annotate_Time,time_when
25,"before leaving, the fairy godmother said she should be home by midnight.","said, should be","the fairy godmother, she",NaN,NaN,home,"before leaving, by midnight",
26,"when cinderella entered the palace, everybody was struck by her beauty.",was struck,everybody,NaN,NaN,NaN,when cinderella entered the palace,when cinderella entered the palace
27,nobody knew who she really was.,"knew, was","Nobody, she",NaN,NaN,NaN,NaN,
28,the handsome prince also saw her.,saw,The handsome prince,her,NaN,NaN,NaN,
29,he fell in love with cinderella.,fell in love,He,Cinderella,NaN,NaN,NaN,


## Q5: location (Where?)
### 1. Extracting clauses with "where" conjunction

In [38]:
# collect words from whole WHERE clauses to where_list
def extract_all_children_where(doc, head):
    if head.children:
        for child in head.children:
            where_list.append(child.text)
            extract_all_children_where(doc, child)

In [39]:
def get_where_clauses(data):

    global where_list
    where_list = []
    where_str = ''

    data['location_where'] = None

    # find collocations which start with WHERE word in each row
    for row in range(data.shape[0]):
        doc = nlp(data.iloc[row,0])

        for word in doc:

            # check if word belongs to WHERE collocation (works if where_list is not empty)
            if word.text in where_list:
                where_str = where_str + ' ' + str(word.text)
            else:
                # if doesn't belong, then collocation is over, clean it
                where_list = []

            # if word = 'where' then run the function that collects words from whole WHERE collocation
            if word.text == 'where':
                where_list.append(word.text)
                head = word.head
                where_list.append(head.text)
                extract_all_children_where(doc, head)
                where_str = 'where'

        data['location_where'][row] = where_str
        where_str = ''

    print('DONE!')

    return data

In [40]:
data_3 = annotate.copy()
data_3["Sentence_content"] = data_3["Sentence_content"].str.lower()
data_3["Annotate_Location"] = data_3["Annotate_Location"].str.lower()
data_3 = get_where_clauses(data_3)
print("Example:")
data_3.iloc[97:99, [0, 7]]

DONE!
Example:


,Sentence_content,location_where
97,she asked the mirror where snow white lived.,where snow white lived
98,"since then, the queen's heart was full of hatred for the young lady that she swore to get rid of her, as soon as possible.",


## [Function] Evaluation of the results (for 'agents', 'action', and 'patients')

In [41]:
def evaluation(df, metric):

    # choose metric, create separate evaluation dataframe for this metric, rename columns
    if metric == 'agents':
        evaluation_df = df.loc[:, ['Annotate_Agent', 'agents_in_sent']]
        evaluation_df = evaluation_df.rename(columns={"Annotate_Agent": "annotate", "agents_in_sent": "result"})
    elif metric == 'actions':
        evaluation_df = df.loc[:, ['Annotate_Action', 'actions_in_sent']]
        evaluation_df = evaluation_df.rename(columns={"Annotate_Action": "annotate", "actions_in_sent": "result"})
    elif metric == 'patients':
        evaluation_df = df.loc[:, ['Annotate_Direct_Obj', 'patients_in_sent']]
        evaluation_df = evaluation_df.rename(columns={"Annotate_Direct_Obj": "annotate", "patients_in_sent": "result"})

    #  convert annotate column from string to list
    evaluation_df['annotate'] = evaluation_df.annotate.str.split(", ")

    #  convert result column from nested list to simple list
    for i in range(evaluation_df.shape[0]):
        evaluation_df['result'][i] = [item for sublist in evaluation_df['result'][i] for item in sublist]
        evaluation_df['result'][i] = list(dict.fromkeys(evaluation_df['result'][i]))

    #  convert non-list elements to list
    for i in range(evaluation_df.shape[0]):
        if type(evaluation_df['result'][i]) != list:
            evaluation_df['result'][i] = []
        if type(evaluation_df['annotate'][i]) != list:
            evaluation_df['annotate'][i] = []
        if None in evaluation_df['result'][i]:
            evaluation_df['result'][i].remove(None)

    evaluation_df['P'] = 0
    evaluation_df['L'] = 0
    evaluation_df['E'] = 0
    evaluation_df['M'] = 0
    evaluation_df['W'] = 0


    for i in range(evaluation_df.shape[0]):
        for r in evaluation_df['result'][i]:
            if r in evaluation_df['annotate'][i]:
                evaluation_df['P'][i] +=1
            else:
                if r in [word for line in evaluation_df['annotate'][i] for word in line.split()]:
                    evaluation_df['L'][i] +=1
                else:
                    evaluation_df['W'][i] +=1
            for a in evaluation_df['annotate'][i]:
                if r != a and a in r.split():
                    evaluation_df['E'][i] +=1

        flag = False
        for a in evaluation_df['annotate'][i]:
            for sub in a.split():
                if sub in evaluation_df['result'][i]:
                    flag = True
            if not flag:
                evaluation_df['M'][i] +=1
            flag = False

    P_score = 0
    L_score = 0
    E_score = 0
    M_score = 0
    W_score = 0

    for i in range(evaluation_df.shape[0]):
        P_score+= evaluation_df['P'][i]
        L_score+= evaluation_df['L'][i]
        E_score+= evaluation_df['E'][i]
        M_score+= evaluation_df['M'][i]
        W_score+= evaluation_df['W'][i]

    print("P: ", P_score, "; L_score: ", L_score, "; E_score: ", E_score, "; M_score: ",M_score, "; W_score: ",W_score)
    Precision = (P_score + L_score + E_score) / (P_score + L_score + E_score + W_score)
    Recall = (P_score + L_score + E_score) / (P_score + L_score + E_score + M_score)
    F1_score = 2 * (Precision * Recall) / (Precision + Recall)
    print('Precision: ', round(Precision, 4), '\n', 'Recall: ', round(Recall, 4), '\n', 'F-score: ', round(F1_score, 4), '\n')

#### [Results] Evaluation for agents

In [42]:
evaluation(data_1, 'agents')

P:  123 ; L_score:  131 ; E_score:  0 ; M_score:  17 ; W_score:  61
Precision:  0.8063 
 Recall:  0.9373 
 F-score:  0.8669 



<ipython-input-41-6f0fecbb1f93>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['P'][i] +=1
<ipython-input-41-6f0fecbb1f93>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['L'][i] +=1
<ipython-input-41-6f0fecbb1f93>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['W'][i] +=1
<ipython-input-41-6f0fecbb1f93>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

#### [Results] Evaluation for actions

In [43]:
evaluation(data_1, 'actions')

P:  171 ; L_score:  81 ; E_score:  0 ; M_score:  28 ; W_score:  57
Precision:  0.8155 
 Recall:  0.9 
 F-score:  0.8557 



<ipython-input-41-6f0fecbb1f93>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['M'][i] +=1
<ipython-input-41-6f0fecbb1f93>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['P'][i] +=1
<ipython-input-41-6f0fecbb1f93>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['L'][i] +=1
<ipython-input-41-6f0fecbb1f93>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

#### [Results] Evaluation for patients

In [44]:
evaluation(data_1, 'patients')

P:  14 ; L_score:  51 ; E_score:  0 ; M_score:  56 ; W_score:  30
Precision:  0.6842 
 Recall:  0.5372 
 F-score:  0.6019 



<ipython-input-41-6f0fecbb1f93>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['M'][i] +=1
<ipython-input-41-6f0fecbb1f93>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['P'][i] +=1
<ipython-input-41-6f0fecbb1f93>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['L'][i] +=1
<ipython-input-41-6f0fecbb1f93>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

## [Function] Evaluation of the results (for 'time' and 'location')

In [45]:
def evaluation_tl(df, metric):

    # choose metric, create separate evaluation dataframe for this metric, rename columns
    if metric == 'time':
        evaluation_df = df.loc[:, ['Annotate_Time', 'time_when']]
        evaluation_df = evaluation_df.rename(columns={"Annotate_Time": "annotate", "time_when": "result"})
    elif metric == 'location':
        evaluation_df = df.loc[:, ['Annotate_Location', 'location_where']]
        evaluation_df = evaluation_df.rename(columns={"Annotate_Location": "annotate", "location_where": "result"})

    #  convert annotate column from string to list
    evaluation_df['annotate'] = evaluation_df.annotate.str.split(", ")

    #  convert result column from nested list to simple list
    for i in range(evaluation_df.shape[0]):
        if  isinstance(evaluation_df['result'][i], (str)):
            evaluation_df['result'][i] = list(evaluation_df['result'][i].split(', '))
        else:
            print(type(evaluation_df['result'][i]))
    #  convert non-list elements to list
    for i in range(evaluation_df.shape[0]):
        if type(evaluation_df['result'][i]) != list:
            evaluation_df['result'][i] = []
        if type(evaluation_df['annotate'][i]) != list:
            evaluation_df['annotate'][i] = []
        if None in evaluation_df['result'][i]:
            evaluation_df['result'][i].remove(None)

    evaluation_df['P'] = 0
    evaluation_df['L'] = 0
    evaluation_df['E'] = 0
    evaluation_df['M'] = 0
    evaluation_df['W'] = 0


    for i in range(evaluation_df.shape[0]):
        for r in evaluation_df['result'][i]:
            if r in evaluation_df['annotate'][i]:
                evaluation_df['P'][i] +=1
            else:
                if r in [word for line in evaluation_df['annotate'][i] for word in line.split()]:
                    evaluation_df['L'][i] +=1
            for a in evaluation_df['annotate'][i]:
                if r != a and a in r.split():
                    evaluation_df['E'][i] +=1
                if a not in r:
                    evaluation_df['M'][i] +=1

    P_score = 0
    L_score = 0
    E_score = 0
    M_score = 0
    W_score = 0

    for i in range(evaluation_df.shape[0]):
        P_score+= evaluation_df['P'][i]
        L_score+= evaluation_df['L'][i]
        E_score+= evaluation_df['E'][i]
        M_score+= evaluation_df['M'][i]
        W_score+= evaluation_df['W'][i]
    print("P: ", P_score, "; L_score: ", L_score, "; E_score: ", E_score, "; M_score: ",M_score, "; W_score: ",W_score)
    Precision = (P_score + L_score + E_score) / (P_score + L_score + E_score + W_score)
    Recall = (P_score + L_score + E_score) / (P_score + L_score + E_score + M_score)
    F1_score = 2 * (Precision * Recall) / (Precision + Recall)
    print('Precision: ', round(Precision, 4), '\n', 'Recall: ', round(Recall, 4), '\n', 'F-score: ', round(F1_score, 4), '\n')



#### [Results] Evaluation for time

In [46]:
evaluation_tl(data_2, 'time')

P:  4 ; L_score:  0 ; E_score:  1 ; M_score:  50 ; W_score:  0
Precision:  1.0 
 Recall:  0.0909 
 F-score:  0.1667 



<ipython-input-45-ed169b5e2dfc>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['M'][i] +=1
<ipython-input-45-ed169b5e2dfc>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['P'][i] +=1
<ipython-input-45-ed169b5e2dfc>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['E'][i] +=1


#### [Results] Evaluation for location

In [47]:
evaluation_tl(data_3, 'location')


P:  1 ; L_score:  0 ; E_score:  0 ; M_score:  58 ; W_score:  0
Precision:  1.0 
 Recall:  0.0169 
 F-score:  0.0333 



<ipython-input-45-ed169b5e2dfc>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['M'][i] +=1
<ipython-input-45-ed169b5e2dfc>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_df['P'][i] +=1
